In [1]:
import os
import sys
import numpy as np
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [2]:
from pyspark import SparkContext, SparkConf
conf = SparkConf()

In [3]:
conf.set("spark.app.name", "ekaterina.kucheryavenko Spark RDD app") 

In [4]:
sc = SparkContext("local", "First App")

In [5]:
# sc.getConf().getAll()

In [6]:
!hdfs dfs -ls /labs/lab06data/ml-100k/u.data

-rw-r--r--   2 hdfs hdfs    1979173 2020-09-30 12:16 /labs/lab06data/ml-100k/u.data


In [7]:
# user id | item id | rating | timestamp
rdd_rating = sc.textFile("/labs/lab06data/ml-100k/u.data")

In [8]:
rdd_rating.take(2)
# rdd_rating.count()

['196\t242\t3\t881250949', '186\t302\t3\t891717742']

In [9]:
# movie id | movie title | release date | video release date |
#               IMDb URL | unknown | Action | Adventure | Animation |
#               Children's | Comedy | Crime | Documentary | Drama | Fantasy |
#               Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
#               Thriller | War | Western |
rdd_title = sc.textFile("/labs/lab06data/ml-100k/u.item")


In [10]:
rdd_title.take(2)
# rdd_title.count()

['1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0',
 '2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0']

In [25]:
# NEW APPROACH

In [29]:
# user id | item id | rating | timestamp
rdd_rating = sc.textFile("/labs/lab06data/ml-100k/u.data")

In [59]:
# Подсчитываем общее количество рейтингов по каждому фильму и сортируем
rdd_rating_all = rdd_rating.map(lambda x : x.split("\t")[2]).sortByKey().countByKey()
rdd_rating_all = dict(rdd_rating_all)
rdd_rating_all = np.array(list(rdd_rating_all.values()))
rdd_rating_all

array([ 6110, 11370, 27145, 34174, 21201])

In [60]:
# Подсчитываем общее количество рейтингов по фильму 191 и сортируем
RDD_mov_rating_191 = rdd_rating.map(lambda x : (x.split("\t")[1],x.split("\t")[2],1)).filter(lambda x: x[0] == '191').map(lambda x: x[1]).sortByKey().countByKey()
RDD_mov_rating_191 = dict(RDD_mov_rating_191)
RDD_mov_rating_191 = np.array(list(RDD_mov_rating_191.values()))
RDD_mov_rating_191

array([  2,   6,  44, 117, 107])

In [61]:
# записываем в json
import json
from json import JSONEncoder
import numpy

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

In [62]:
numpyData = {"hist_film": RDD_mov_rating_191, "hist_all": rdd_rating_all}
# encodedNumpyData = json.dumps(numpyData, cls=NumpyArrayEncoder)  # use dump() to write array into file
with open("../lab06.json", "w") as write_file:
    json.dump(numpyData, write_file, cls=NumpyArrayEncoder)

In [63]:
sc.stop()